In [22]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import librosa, glob, sys, pickle, random, re, os, csv, pathlib, ast, os.path, json
import pandas as pd
from statistics import mean, variance
from tqdm import tqdm
speakers = [f.name for f in os.scandir('./') if f.is_dir() and 'english' in f.name]
print(speakers)


['tamil_male_english', 'gujarati_female_english', 'kannada_male_english', 'malayalam_male_english', 'hindi_male_english', 'manipuri_female_english', 'assamese_female_english', 'rajasthani_male_english']


In [10]:
for speaker in speakers:
    all_file_path = "{}/manifests/quartznet_outputs/all.txt".format(speaker)
    
    ref_hyp_file = open(all_file_path, 'r')
    paragraphs = ref_hyp_file.read().strip().split('\n\n')
    hyp_list = [para.strip().split('\n')[4][5:] for para in paragraphs]
    
    all_json_path = "{}/manifests/all.json".format(speaker)
    all_json = open(all_json_path, 'r')
    samples = all_json.readlines()
    
    new_json_file = open("{}/all.json".format(speaker), 'w')
    new_json_samples = []
    for sample, hyp in zip(samples, hyp_list):
        new_dict = json.loads(sample)
        new_dict['pseudo_text'] = hyp
        json.dump(new_dict, new_json_file)
        new_json_file.write('\n')

In [11]:
!wc -l */all.json
!wc -l */manifests/all.json

    8378 assamese_female_english/all.json
    4670 gujarati_female_english/all.json
    5131 hindi_male_english/all.json
    4777 kannada_male_english/all.json
    5130 malayalam_male_english/all.json
   10035 manipuri_female_english/all.json
    4804 rajasthani_male_english/all.json
    7546 tamil_male_english/all.json
   50471 total
    8378 assamese_female_english/manifests/all.json
    4670 gujarati_female_english/manifests/all.json
    5131 hindi_male_english/manifests/all.json
    4777 kannada_male_english/manifests/all.json
    5130 malayalam_male_english/manifests/all.json
   10035 manipuri_female_english/manifests/all.json
    4809 rajasthani_male_english/manifests/all.json
    7546 tamil_male_english/manifests/all.json
   50476 total


In [13]:
speaker = "rajasthani_male_english"
all_file_path = "{}/manifests/quartznet_outputs/all.txt".format(speaker)
    
ref_hyp_file = open(all_file_path, 'r')
paragraphs = ref_hyp_file.read().strip().split('\n\n')
hyp_list = [para.strip().split('\n')[4][5:] for para in paragraphs]
    
all_json_path = "{}/manifests/all.json".format(speaker)
all_json = open(all_json_path, 'r')
samples = all_json.readlines()
    
new_json_file = open("{}/all.json".format(speaker), 'w')
new_json_samples = []
for sample, hyp in zip(samples, hyp_list):
    new_dict = json.loads(sample)
    new_dict['pseudo_text'] = hyp
    json.dump(new_dict, new_json_file)
    new_json_file.write('\n')

In [14]:
!wc -l */all.json
!wc -l */manifests/all.json

    8378 assamese_female_english/all.json
    4670 gujarati_female_english/all.json
    5131 hindi_male_english/all.json
    4777 kannada_male_english/all.json
    5130 malayalam_male_english/all.json
   10035 manipuri_female_english/all.json
    4809 rajasthani_male_english/all.json
    7546 tamil_male_english/all.json
   50476 total
    8378 assamese_female_english/manifests/all.json
    4670 gujarati_female_english/manifests/all.json
    5131 hindi_male_english/manifests/all.json
    4777 kannada_male_english/manifests/all.json
    5130 malayalam_male_english/manifests/all.json
   10035 manipuri_female_english/manifests/all.json
    4809 rajasthani_male_english/manifests/all.json
    7546 tamil_male_english/manifests/all.json
   50476 total


In [23]:
def _write(json_path, items, file):
    with open(json_path+file, 'w') as f:
        for item in tqdm(items):
            json.dump(item, f)
            f.write('\n')

In [28]:
for speaker in speakers:
    json_path = speaker + '/'
    with open(json_path+'all.json', 'r') as jd:
        files = jd.readlines()
        files = [json.loads(file) for file in files]
        
        random.seed(42)
        random.shuffle(files)
        
        num_samples = len(files)
        p70, p97, p97_50 = int(num_samples*.7), int(num_samples*.97), int(num_samples*.97)+50
        
        assert p97_50 <= num_samples-50
        selection, test, dev, seed = files[:p70], files[p70:p97], files[p97:p97_50], files[num_samples-50:]
        seed_plus_dev = seed+dev
        
        print("total, selection, seed, dev, seed_plus_dev, test for {}".format(speaker))
        print(list(map(len, [files, selection, seed, dev, seed_plus_dev, test])))
        _write(json_path, seed, 'seed.json')
        _write(json_path, dev, 'dev.json')
        _write(json_path, seed_plus_dev, 'seed_plus_dev.json')
        _write(json_path, test, 'test.json')
        _write(json_path, selection, 'selection.json')

total, selection, seed, dev, seed_plus_dev, test for tamil_male_english
[7546, 5282, 50, 50, 100, 2037]


100%|██████████| 5282/5282 [00:00<00:00, 51243.75it/s]


total, selection, seed, dev, seed_plus_dev, test for gujarati_female_english
[4670, 3269, 50, 50, 100, 1260]


100%|██████████| 3269/3269 [00:00<00:00, 57020.63it/s]


total, selection, seed, dev, seed_plus_dev, test for kannada_male_english
[4777, 3343, 50, 50, 100, 1290]


...

100%|██████████| 7024/7024 [00:00<00:00, 56503.03it/s]


total, selection, seed, dev, seed_plus_dev, test for assamese_female_english
[8378, 5864, 50, 50, 100, 2262]


100%|██████████| 5864/5864 [00:00<00:00, 55168.53it/s]


total, selection, seed, dev, seed_plus_dev, test for rajasthani_male_english
[4809, 3366, 50, 50, 100, 1298]


100%|██████████| 3366/3366 [00:00<00:00, 56345.22it/s]


In [29]:
!wc -l */*.json

    8378 assamese_female_english/all.json
      50 assamese_female_english/dev.json
      50 assamese_female_english/seed.json
     100 assamese_female_english/seed_plus_dev.json
    5864 assamese_female_english/selection.json
    2262 assamese_female_english/test.json
    4670 gujarati_female_english/all.json
      50 gujarati_female_english/dev.json
      50 gujarati_female_english/seed.json
     100 gujarati_female_english/seed_plus_dev.json
    3269 gujarati_female_english/selection.json
    1260 gujarati_female_english/test.json
    5131 hindi_male_english/all.json
      50 hindi_male_english/dev.json
      50 hindi_male_english/seed.json
     100 hindi_male_english/seed_plus_dev.json
    3591 hindi_male_english/selection.json
    1386 hindi_male_english/test.json
    4777 kannada_male_english/all.json
      50 kannada_male_english/dev.json
      50 kannada_male_english/seed.json
     100 kannada_male_english/seed_plus_dev.json
    3343 kannada_male_english/selection.json
    1290